In [8]:
import xml.etree.ElementTree as ET
import json

index=4

nodes=[]
ways=[]
relations=[]

def process_element(el):
    if el.tag=="node":
        nodes.append({
            "id": int(el.attrib['id']),
            "lat": float(el.attrib['lat']),
            "lon": float(el.attrib['lon']),
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el}
        })
    if el.tag=="way":
        ways.append({
            "id": int(el.attrib['id']),
            "nodes": [int(nd.attrib["ref"]) for nd in el if nd.tag=="nd"],
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el if tag.tag=="tag"}
        }) 
    if el.tag=="relation":
        relations.append({
            "id": int(el.attrib['id']),
            "nodes": [int(nd.attrib["ref"]) for nd in el if nd.tag=="member" and nd.attrib["type"]=="node"],
            "ways": [int(nd.attrib["ref"]) for nd in el if nd.tag=="member" and nd.attrib["type"]=="way"],
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in el if tag.tag=="tag"}
        })

# Parse the XML file
with open('split/kansai-split'+str(index)+'.osm', 'rb') as file:
    context = ET.iterparse(file, events=('start', 'end'))

    # Iterate over the elements
    counter = 0
    for event, element in context:
        if event == 'start':
            process_element(element)
            element.clear()
            counter += 1
            if counter % 100000 == 0:
                print(float(counter)/105425265.0*25, "%")
    

with open('data/nodes'+str(index)+'.json', 'w') as file:
    json.dump(nodes, file)
with open('data/ways'+str(index)+'.json', 'w') as file:
    json.dump(ways, file)
with open('data/relations'+str(index)+'.json', 'w') as file:
    json.dump(relations, file)

0.023713480824544286 %
0.04742696164908857 %
0.07114044247363285 %
0.09485392329817714 %
0.11856740412272145 %
0.1422808849472657 %
0.16599436577181 %
0.18970784659635428 %
0.21342132742089862 %
0.2371348082454429 %
0.26084828906998714 %
0.2845617698945314 %
0.30827525071907574 %
0.33198873154362 %
0.3557022123681643 %
0.37941569319270857 %
0.40312917401725284 %
0.42684265484179723 %
0.4505561356663415 %
0.4742696164908858 %
0.49798309731543006 %
0.5216965781399743 %
0.5454100589645186 %
0.5691235397890628 %
0.5928370206136072 %
0.6165505014381515 %
0.6402639822626958 %
0.66397746308724 %
0.6876909439117843 %
0.7114044247363286 %
0.7351179055608729 %


0.7588313863854171 %
0.7825448672099614 %
0.8062583480345057 %
0.82997182885905 %
0.8536853096835945 %
0.8773987905081387 %
0.901112271332683 %
0.9248257521572273 %
0.9485392329817716 %
0.9722527138063158 %
0.9959661946308601 %
1.0196796754554043 %
1.0433931562799486 %
1.0671066371044928 %
1.090820117929037 %
1.1145335987535814 %
1.1382470795781257 %
1.16196056040267 %
1.1856740412272144 %
1.2093875220517587 %
1.233101002876303 %
1.2568144837008473 %
1.2805279645253915 %
1.3042414453499358 %
1.32795492617448 %
1.3516684069990244 %
1.3753818878235686 %
1.399095368648113 %
1.4228088494726572 %
1.4465223302972015 %
1.4702358111217457 %
1.49394929194629 %
1.5176627727708343 %
1.5413762535953788 %
1.5650897344199228 %
1.588803215244467 %
1.6125166960690114 %
1.6362301768935557 %
1.6599436577181 %
1.6836571385426442 %


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f1dcc541c90>>
Traceback (most recent call last):
  File "/home/yl/Documents/Kobe/OpenData/national-roads/env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


1.707370619367189 %
1.7310841001917332 %
1.7547975810162775 %
1.7785110618408217 %
1.802224542665366 %
1.8259380234899103 %
1.8496515043144546 %
1.8733649851389989 %
1.8970784659635431 %
1.9207919467880874 %
1.9445054276126317 %
1.968218908437176 %
1.9919323892617202 %
2.0156458700862645 %
2.0393593509108086 %
2.063072831735353 %
2.086786312559897 %
2.1104997933844416 %
2.1342132742089857 %
2.15792675503353 %
2.181640235858074 %
2.2053537166826187 %
2.2290671975071628 %
2.2527806783317073 %
2.2764941591562513 %
2.300207639980796 %
2.32392112080534 %
2.3476346016298844 %
2.371348082454429 %
2.3950615632789733 %
2.4187750441035174 %
2.442488524928062 %
2.466202005752606 %
2.4899154865771504 %
2.5136289674016945 %
2.537342448226239 %
2.561055929050783 %
2.5847694098753276 %
2.6084828906998716 %
2.632196371524416 %
2.65590985234896 %
2.6796233331735047 %
2.7033368139980487 %
2.727050294822593 %
